# Thinking about data probabilistically

In [ ]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
from myst_nb import glue

true = 12
np.random.seed(1)
mu = 10.4
sigma = 1.6
n = norm(mu, sigma)
x = np.linspace(mu - 4 * sigma, mu + 4 * sigma, 1000)
fig0, ax0 = plt.subplots(figsize=(5,3))
ax0.plot(x, n.pdf(x))
ax0.set_ylim(0, None)
ax0.set_xlim(x.min(), x.max())
ax0.set_xlabel('$x$')
ax0.set_ylabel('$p(x)$')
fig0.tight_layout()
glue("normal_fig", fig0, display=False)

Every measurement has some uncertainty associated with it -- no measurement is perfect. 
What causes this uncertainty is not under consideration now, but rather what *uncertainty* means and how we can think about it in our analysis. 
Typically, a value with an uncertainty is written as $\mu \pm \sigma$, where $\mu$ and $\sigma$ are the measurement and uncertainty, respectively. 
While there is (unfortunately) no standard for what this nomenclature means, a common interpretation is that this is describing a normal distribution, which is centred at $\mu$ with a standard deviation of $\sigma$ ({numref}`normal`). 

```{glue:figure} normal_fig
:name: "normal"

A normal distribution (blue line), centred on 10.4 with a standard deviation of 1.6.
```

Consider the mathematical model $x=a^2$, that provides a model dataset (of a single data point), where $a$ is the optimisation parameter[^1].
Assume further that we experimentally measured $x$ to be $x = \mu \pm \sigma = 10.4 \pm 1.6$ as shown in {numref}`normal`.
The aim of model-dependent analysis is to find the value of $a$ that leads to the best agreement between our mathematical model and the experimental dataset.
Since we assume the measured value to be normally distributed, its probability distribution is
```{math}
:label: normal
p(x) = \frac{1}{\sigma\sqrt{2\pi}}\exp{\bigg[-\frac{1}{2}\Big(\frac{x-\mu}{\sigma}\Big)^2\bigg]}.
```
$p$ is called the likelihood of the observation $\mu \pm \sigma$ given the parameter $a$.
To get the best agreement between the model and the experiment, we need to maximise (or, more commonly, minimise the negative) likelihood. 
Before passing this task to an optimisation algorithm, we need to have a function that calculates $-p(x)$ from $a$, for Eqn. {eq}`normal`.

In [ ]:
import numpy as np

mu = 10.4
sigma = 1.6

def nl(a):
    """
    Calculate the negative likelihood for a normal distribution.
    
    Parameters
    ----------
    a: 
        The optimisation parameter, a.
    
    Returns
    -------
    :
        The negative likelihood.
    """
    x = a ** 2
    likelihood = 1 / (sigma * np.sqrt(2 * np.pi)) * np.exp(-0.5 * ((x - mu) / sigma) ** 2)
    return -likelihood

The above function calculates the negative likelihood of a given $a$ for the data shown in {numref}`normal`.
We can then use the `scipy.optimize.minimize` optimisation algorithm to minimise this, other minimization libraries are available in Python. 

In [ ]:
from scipy.optimize import minimize

result = minimize(nl, 2)
print(result)

The optimisation was successful.

In [ ]:
result.success

````{margin}
```{note}
This result shouldn't be surprising as $\sqrt{10.4} \approx 3.225$.
```
````
The optimised value of $a$ was found to be 3.225. 

In [ ]:
result.x

In [ ]:
x = np.linspace(mu - 4 * sigma, mu + 4 * sigma, 1000)
fig1, ax1 = plt.subplots(figsize=(5,3))
ax1.plot(x, n.pdf(x))
ax1.plot(3.22490293 ** 2, n.pdf(3.22490293 ** 2), 'ro')
ax1.set_ylim(0, None)
ax1.set_xlim(x.min(), x.max())
ax1.set_xlabel('$x$')
ax1.set_ylabel('$p(x)$')
fig1.tight_layout()
glue("normal_fit_fig", fig1, display=False)

x = np.linspace(1, 10, 5)
y = 4.1 * x + 2.3
y_noise = np.random.randn(x.size) * 2
y += y_noise
dy = np.abs(np.random.randn() * 2)

fig2, ax2 = plt.subplots(1, 2, figsize=(10,3))
ax2[0].errorbar(x, y, dy, marker='.', ls='')
ax2[0].set_ylim(0, None)
ax2[0].set_xlabel('$x$')
ax2[0].set_ylabel('$y$')
y_range = np.arange(-5, 60, 0.1)
for i, yy in enumerate(y):
    ax2[1].fill_between(y_range, norm(yy, dy).pdf(y_range), color='b', alpha=0.08 * (i + 1), lw=0)
ax2[1].set_xlim(y_range.min(), y_range.max())
ax2[1].set_ylim(0, None)
ax2[1].set_xlabel('$y$')
ax2[1].set_ylabel('$p(y)$')
fig2.tight_layout()
glue("multid_fig", fig2, display=False)

from scipy.stats import linregress

result = linregress(x, y)

fig3, ax3 = plt.subplots(1, 2, figsize=(10,3))
ax3[0].errorbar(x, y, dy, marker='.', ls='')
ax3[0].plot(x, x * result.slope + result.intercept, 'g-')
ax3[0].set_ylim(0, None)
ax3[0].set_xlabel('$x$')
ax3[0].set_ylabel('$y$')

y_range = np.arange(-5, 60, 0.1)
for i, yy in enumerate(y):
    ax3[1].fill_between(y_range, norm(yy, dy).pdf(y_range), color='b', alpha=0.08 * (i + 1), lw=0)
    ax3[1].plot(x[i] * result.slope + result.intercept, norm(yy, dy).pdf(x[i] * result.slope + result.intercept), 'go')
ax3[1].set_xlim(y_range.min(), y_range.max())
ax3[1].set_ylim(0, None)
ax3[1].set_xlabel('$y$')
ax3[1].set_ylabel('$p(y)$')
fig3.tight_layout()
glue("multid_fit_fig", fig3, display=False)

This result is plotted on top of the data in {numref}`normal_fit`.

```{glue:figure} normal_fit_fig
:name: "normal_fit"

A normal distribution (blue line), centred on 10.4 with a standard deviation of 1.6 with the maximum likelihood value (red circle).
```

Consider extending this beyond datasets with just a single data point, where each data point is itself a normal distribution. 
This is visualised in {numref}`multid` for five data points; the plot on the left-hand-side presents the data in (hopefully) a familiar way, while the right-hand-side shows the view as though one is sat on the plane of the computer screen, looking along the *x*-axis. 

```{glue:figure} multid_fig
:name: "multid"

Plots with more data points; the standard way to plot data with some uncertainty (e.g., points with error bars) (LHS) and the view of the five likelihood functions for each data point (note that here the uncertainty in each data point is taken to be the same, i.e., it is homoscedastic) (RHS).
```

The data in {numref}`multid` appears linear, therefore a "straight line fit" can be performed -- this is where a gradient and intercept for a straight line (the mathematical model) is found. 
This results in a model that maximises the product of the likelihoods for each data point, as best as possible given the constraint that the model must hold. 
It is clear in {numref}`multid_fit` that the green model cannot maximise any of the individual distributions, but overall, this is the best possible agreement for the distributions. 

```{glue:figure} multid_fit_fig
:name: "multid_fit"

{numref}`multid` with a fitted linear model (green).
```

This way of thinking about experimental data can be carried through to the use of model-dependent analysis.

[^1]: This model has no real physical rationality, but is only a representative example. 